In [1]:
import ee
import geemap

In [2]:
ee.Initialize()

In [9]:
import geopandas as gpd

shapefile = gpd.read_file("ga-boundary/ga.shp")
shp = shapefile.geometry[0]
xs, ys = shp.exterior.coords.xy
shp_list = [[x, y] for x, y in zip(xs, ys)]
roi = ee.Geometry.Polygon(shp_list, None, False)

In [4]:
def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi

In [5]:
ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
bacon_shp = ga_counties[ga_counties["NAME10"] == "Bacon"].geometry

In [10]:
bacon_shp = ga_counties[ga_counties["NAME10"] == "Bacon"].geometry.values[0]
bacon_roi = pd_shp_to_ee_poly(bacon_shp)

In [11]:
bands = ['Red', 'Green', 'Blue']
roi = bacon_roi.simplify(maxError = 1)
fname = 'vids/georgia-bacon.gif'

collection = geemap.landsat_timeseries(
    roi = roi, 
    start_year = 1985, end_year = 2019, 
    start_date = '06-10', end_date = '09-20'
)

video_args = {
    'dimensions': 768,
    'region': roi,
    'framesPerSecond': 5,
    'bands': bands,
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1]
}

geemap.download_ee_video(collection, video_args, fname)
geemap.add_text_to_gif(
    fname, fname, xy=('92%', '3%'), 
    text_sequence = 1985, font_size = 20, 
    font_color = '#ffffff', 
    add_progress_bar = False
)
geemap.show_image(fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-bacon.gif


Output()

In [34]:
crop_values = {
    242: 'blueberries',
    67: 'peaches',
    72: 'citrus',
    211: 'olives'
}

crop_numbers = list(crop_values.keys())

collection = ee.ImageCollection('USDA/NASS/CDL') \
    .filter(ee.Filter.date('1997-01-01', '2021-01-01')) \
    .select('cropland') \
    .filter(ee.Filter.inList('cropland_class_names', crop_numbers))

collection.size().getInfo()

# collection = geemap.create_timeseries(
#     collection = 'USDA/NASS/CDL',
#     start_date = '1997-01-01',
#     end_date = '2021-01-01',
#     region = roi,
#     bands = ['cropland']
# )

0

In [29]:
collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'cropland',
     'data_type': {'type': 'PixelType',
      'precision': 'double',
      'min': 0,
      'max': 255},
     'dimensions': [1, 1],
     'origin': [-83, 31],
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1009843200000,
    'system:footprint': {'geodesic': False,
     'type': 'Polygon',
     'coordinates': [[[-82.53515556543397, 31.672588041078466],
       [-82.52506856215776, 31.672549041626322],
       [-82.52334956133085, 31.672510041641484],
       [-82.52139656095234, 31.67251804177463],
       [-82.52141156164367, 31.67350404150243],
       [-82.52138856146819, 31.673994042190948],
       [-82.5215415617398, 31.685003044991756],
       [-82.52158756287749, 31.686762044875117],
       [-82.52157256170324, 31.68742204544948],
       [-82.52171356259639, 31.69817704713942],
       [-82.52170156249271, 31.699110046927263

In [23]:

#filter(
#    ee.Filter.inList('cropland_class_names', ee.List(crop_numbers))
#)

roi = bacon_roi.simplify(maxError = 1)
fname = 'vids/georgia-bacon-crops.gif'

video_args = {
    'dimensions': 768,
    'region': roi,
    'framesPerSecond': 5,
    'forceRgbOutput' : True
}

geemap.download_ee_video(collection, video_args, fname)
# geemap.add_text_to_gif(
#     fname, fname, xy=('92%', '3%'), 
#     text_sequence = 1997, font_size = 20, 
#     font_color = '#ffffff', 
#     add_progress_bar = False
# )
geemap.show_image(fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-bacon-crops.gif


Output()

In [9]:
def export_video(roi, collection, bands, fname):
    video_args = {
        'dimensions': 768,
        'region': roi,
        'framesPerSecond': 5,
        'bands': bands,
        'min': 0,
        'max': 4000,
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 1985, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )
    geemap.show_image(fname)

In [10]:
collection = geemap.landsat_timeseries(
    roi = roi, 
    start_year = 1985, end_year = 2019, 
    start_date = '06-10', end_date = '09-20'
)

bands = ['Red', 'Green', 'Blue']
fname = 'vids/georgia-ts-summer.gif'

export_video(collection, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-summer.gif


Output()

In [11]:
bands = ['NIR', 'Red', 'Green']
fname = 'vids/georgia-ts-vegeation.gif'

export_video(collection, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-vegeation.gif


Output()

In [13]:
bands = ['SWIR1', 'NIR', 'Blue']
fname = 'vids/georgia-ts-agriculture.gif'

export_video(collection, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-agriculture.gif


Output()

In [7]:
collection = geemap.landsat_timeseries(
    roi = roi, 
    start_year = 1985, end_year = 2019, 
    start_date = '01-01', end_date = '03-28'
)

bands = ['Red', 'Green', 'Blue']
fname = 'vids/georgia-ts-winter.gif'

export_video(collection, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-winter.gif


Output()

In [24]:
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG#bands
# ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")
import pandas as pd

collection_dmsp = geemap.create_timeseries(
    collection = "NOAA/DMSP-OLS/NIGHTTIME_LIGHTS",
    region = roi,
    frequency = 'month',
    start_date = '2003-01-01',
    end_date = '2012-04-01'
)

collection_virrs = geemap.create_timeseries(
    collection = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG",
    region = roi, 
    frequency = 'month',
    start_date = '2012-04-01', 
    end_date = '2021-05-01'
)

video_args_dmsp = {
    'dimensions': 500,
    'region': roi,
    'framesPerSecond': 7,
    'bands': ['stable_lights'],
    'min': 0,
    'max': 60,
    'bestEffort': True,
    'palette': ['000000', 'FFFFFF']
}

video_args_virrs = {
    'dimensions': 500,
    'region': roi,
    'framesPerSecond': 7,
    'bands': ['avg_rad'],
    'min': 0,
    'max': 60,
    'bestEffort': True,
    'palette': ['000000', 'FFFFFF']
}

geemap.download_ee_video(collection_dmsp, video_args_dmsp, 'vids/georgia-ts-nighttime-1.gif')
geemap.download_ee_video(collection_virrs, video_args_virrs, 'vids/georgia-ts-nighttime-2.gif')

dates_dmsp = pd.date_range(start = "2003-01", end = "2012-04", freq = "M").strftime('%Y-%m').tolist()
dates_virrs = pd.date_range(start = "2012-04", end = "2021-05", freq = "M").strftime('%Y-%m').tolist()

geemap.add_text_to_gif(
    'vids/georgia-ts-nighttime-1.gif', 'vids/georgia-ts-nighttime-1.gif', xy=('85%', '3%'), 
    text_sequence = dates_dmsp[:-1], font_size = 20, 
    font_color = '#ffffff', 
    add_progress_bar = False
)

geemap.add_text_to_gif(
    'vids/georgia-ts-nighttime-2.gif', 'vids/georgia-ts-nighttime-2.gif', xy=('85%', '3%'), 
    text_sequence = dates_virrs[:-1], font_size = 20, 
    font_color = '#ffffff', 
    add_progress_bar = False
)

#geemap.merge_gifs(['vids/georgia-ts-nighttime-1.gif', 'vids/georgia-ts-nighttime-2.gif'], 'vids/georgia-ts-nighttime.gif')
#geemap.show_image('vids/georgia-ts-nighttime')

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-nighttime-1.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/georgia-ts-nighttime-2.gif
The length of the text sequence must be equal to the number (10) of frames in the gif.


sh: 1: gifsicle: not found


Output()

In [ ]:
# Look at bacon county
# Visualize only blueberry, 
# https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL#bands
# 

Some questions to explore:
- Blueberry farming changing to peach farming - find some areas where this has been happening, look
at them a lot more closely
- 
- Atlanta - zoom into Atlanta and the surrounding counties - changes over time?
- Focus onto the coast: changing coastline?
- Look at rivers for any shrinkage: Flint, Ogeechee, Satilla, St. Marys, and Suwannee

Todo list:
- Get regions of interest (Atlanta and surrounding counties, Bacon county, counties on the ocean) 
- Figure out how to do overlay of different counties on the visualizations; 
see if these can be exported as .mp4 with more pixels
- Understand spatial resolution (maybe zoom and look at the AJC office to figure this out, and
also around Morningside Heights too)
- ACS: look at the demographics with Bacon county over time, also Atlanta/surrounding counties